In [35]:

import cv2                
import numpy as np
import matplotlib.pyplot as plt    
import random

#from extract_bottleneck_features import *
from glob import glob

from keras.applications.resnet50 import decode_predictions, preprocess_input, ResNet50
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint  
from keras.layers import Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing import image                  

from sklearn.datasets import load_files       

from PIL import ImageFile
from tqdm import tqdm

def extract_Xception(tensor):
    from keras.applications.xception import Xception, preprocess_input
    return Xception(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

%matplotlib inline

print("All revv'ed up and ready to go!")


All revv'ed up and ready to go!


In [3]:
# define function to load train, test, and validation datasets
NUM_CATEGORIES = 3

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), NUM_CATEGORIES)
    return files, targets

# load train, test, and validation datasets
base_folder = 'data'
valid_files, valid_targets = load_dataset('{}/valid'.format(base_folder))

# load list of disease names
disease_names = [item.split("/")[2] for item in sorted(glob("{}/valid/*/".format(base_folder)))]


In [4]:
#for item in sorted(glob("{}/valid/*/".format(base_folder))):
#    print()

print(disease_names)

print('Examples of file {} and target {}'.format(valid_files[-5:], valid_targets[-5:]))

#print(valid_files[-5:])
#print(valid_targets[-5:])

print('There are %d total disease categories.' % len(disease_names))
print('There are %d validation disease images.' % len(valid_files))


['melanoma', 'nevus', 'seborrheic_keratosis']
Examples of file ['data/valid/melanoma/ISIC_0013644.jpg' 'data/valid/nevus/ISIC_0015443.jpg'
 'data/valid/nevus/ISIC_0012313.jpg'
 'data/valid/seborrheic_keratosis/ISIC_0012720.jpg'
 'data/valid/nevus/ISIC_0007332.jpg'] and target [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]]
There are 3 total disease categories.
There are 150 validation disease images.


In [5]:

# These are separated as I am using my computer to setup the framework, and AWS for training
# To save space, I didn't download train/test, a total of 10GB as zip!

train_files, train_targets = load_dataset('{}/train'.format(base_folder))
test_files, test_targets = load_dataset('{}/test'.format(base_folder))

print('There are %s total disease images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training disease images.' % len(train_files))
print('There are %d test disease images.'% len(test_files))


There are 2750 total disease images.

There are 2000 training disease images.
There are 600 test disease images.


In [6]:

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


In [7]:
                           
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

print("All done, be sure to run this just once (per Jupyter notebook session)!")


100%|██████████| 600/600 [03:47<00:00,  2.44it/s]


All done, be sure to run this just once (per Jupyter notebook session)!


In [ ]:
%%time

## No beuno
#bottleneck_features = np.load('bottleneck_features/DogXceptionData.npz')
#train_Xception = bottleneck_features['train']
#valid_Xception = bottleneck_features['valid']
#test_Xception = bottleneck_features['test']

print('Converting valid tensors')
valid_Xception = np.array([extract_Xception(np.expand_dims(cur_tensor, axis=0)) for cur_tensor in valid_tensors])
print('Converting train tensors')
train_Xception = np.array([extract_Xception(np.expand_dims(cur_tensor, axis=0)) for cur_tensor in train_tensors])
print('Converting test tensors')
test_Xception = np.array([extract_Xception(np.expand_dims(cur_tensor, axis=0)) for cur_tensor in test_tensors])


#for cur_tensor in valid_tensors:
#    print(cur_tensor[0][0][0])
    #valid_Xception.append()

#valid_Xception = np.array(valid_Xception)

print('ALLL LDOOOOOONE>...')

Converting valid tensors


In [8]:


Xception_model = Sequential()
Xception_model.add(GlobalAveragePooling2D(input_shape = train_Xception.shape[1:]))
Xception_model.add(Dense(len(disease_names), activation = 'softmax'))

Xception_model.summary()

Xception_model.compile(loss = 'categorical_crossentropy'
                        , metrics = ['accuracy']
                        , optimizer = 'rmsprop')

Xception_model.compile(loss = 'categorical_crossentropy'
                        , metrics = ['accuracy']
                        , optimizer = 'rmsprop')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 6147      
Total params: 6,147.0
Trainable params: 6,147.0
Non-trainable params: 0.0
_________________________________________________________________


In [19]:
print(train_Xception.shape[1:])
print(valid_Xception.shape[1:])
print(valid_targets.shape[1:])

print(type(train_Xception))
print(type(np.array([])))

print(len(valid_Xception))
print(len(valid_targets))

print(len(train_Xception))
print(len(train_targets))

(7, 7, 2048)
(7, 7, 2048)
(3,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
835
150
6680
2000


In [9]:

best_Xception_model_file = 'saved_models/weights.best.Xception.hdf5'
    
checkpointer = ModelCheckpoint(filepath = best_Xception_model_file
                               , save_best_only = True
                               , verbose = 1)

Xception_model.fit(train_Xception, train_targets 
                   , batch_size = 20
                   , callbacks = [checkpointer]
                   , epochs = 20
                   , validation_data = (valid_Xception, valid_targets)
                   , verbose = 1
                   )


ValueError: Input arrays should have the same number of samples as target arrays. Found 6680 input samples and 2000 target samples.

In [ ]:

Xception_model.load_weights(best_Xception_model_file)

# get index of predicted dog breed for each image in test set
Xception_predictions = [np.argmax(Xception_model.predict(np.expand_dims(feature, axis=0))) for feature in test_Xception]

# report test accuracy
test_accuracy = 100*np.sum(np.array(Xception_predictions)==np.argmax(test_targets, axis=1))/len(Xception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)


In [ ]:

def disease(image_path):
    img_tensor = path_to_tensor(image_path)
    feature = extract_Xception(img_tensor)
    #print(feature)
    #print(feature.shape)
    index = np.argmax(Xception_model.predict(feature)) #np.expand_dims(feature, axis=0)))
    
    return disease_names[index]

disease('./data/valid/melanoma/ISIC_0012099.jpg')
disease('./data/valid/nevus/ISIC_0001769.jpg')
disease('./data/valid/seborrheic_keratosis/ISIC_0012143.jpg')
